In [ ]:
# =============================================================================
# Training and plotting of final ANN
# =============================================================================

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# empty list to append metric values
mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# 3 months
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size = 0.15, shuffle=False)

# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor

# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40, 
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.1))
    for layer in range(n_hidden):
        model.add(Dense(units = n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = 10)

hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 100, epochs = 5000)
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)
# Save


results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})



C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/5000
5/5 [==============================] - 0s 2ms/step - loss: 13459.5312 - mse: 13459.5312 - mae: 110.1157
Epoch 2/5000
5/5 [==============================] - 0s 2ms/step - loss: 13429.8574 - mse: 13429.8574 - mae: 109.9799
Epoch 3/5000
5/5 [==============================] - 0s 2ms/step - loss: 13394.3154 - mse: 13394.3154 - mae: 109.8197
Epoch 4/5000
5/5 [==============================] - 0s 2ms/step - loss: 13345.1113 - mse: 13345.1113 - mae: 109.5953
Epoch 5/5000
5/5 [==============================] - 0s 2ms/step - loss: 13281.0498 - mse: 13281.0498 - mae: 109.3015
Epoch 6/5000
5/5 [==============================] - 0s 3ms/step - loss: 13191.2412 - mse: 13191.2412 - mae: 108.8961
Epoch 7/5000
5/5 [==============================] - 0s 2ms/step - loss: 13071.4277 - mse: 13071.4277 - mae: 108.3511
Epoch 8/5000
5/5 [==============================] - 0s 3ms/step - loss: 12901.1172 - mse: 12901.1172 - mae: 107.5510
Epoch 9/5000
5/5 [==============================] - 0s 3ms/step 

5/5 [==============================] - 0s 3ms/step - loss: 1648.0643 - mse: 1648.0643 - mae: 27.4312
Epoch 72/5000
5/5 [==============================] - 0s 3ms/step - loss: 1795.0969 - mse: 1795.0969 - mae: 28.5106
Epoch 73/5000
5/5 [==============================] - 0s 3ms/step - loss: 1646.4073 - mse: 1646.4073 - mae: 26.8578
Epoch 74/5000
5/5 [==============================] - 0s 3ms/step - loss: 1801.6647 - mse: 1801.6647 - mae: 28.4331
Epoch 75/5000
5/5 [==============================] - 0s 3ms/step - loss: 1803.6901 - mse: 1803.6901 - mae: 29.3740
Epoch 76/5000
5/5 [==============================] - 0s 3ms/step - loss: 1706.4802 - mse: 1706.4802 - mae: 28.2500
Epoch 77/5000
5/5 [==============================] - 0s 4ms/step - loss: 1643.6002 - mse: 1643.6002 - mae: 27.0173
Epoch 78/5000
5/5 [==============================] - 0s 3ms/step - loss: 1624.5607 - mse: 1624.5607 - mae: 27.0906
Epoch 79/5000
5/5 [==============================] - 0s 3ms/step - loss: 1654.3014 - mse: 1654

Epoch 141/5000
5/5 [==============================] - 0s 3ms/step - loss: 1582.7070 - mse: 1582.7070 - mae: 26.2628
Epoch 142/5000
5/5 [==============================] - 0s 3ms/step - loss: 1602.5409 - mse: 1602.5409 - mae: 26.6631
Epoch 143/5000
5/5 [==============================] - 0s 3ms/step - loss: 1676.9565 - mse: 1676.9565 - mae: 27.3309
Epoch 144/5000
5/5 [==============================] - 0s 3ms/step - loss: 1546.0470 - mse: 1546.0470 - mae: 26.2299
Epoch 145/5000
5/5 [==============================] - 0s 3ms/step - loss: 1601.9733 - mse: 1601.9733 - mae: 26.0066
Epoch 146/5000
5/5 [==============================] - 0s 3ms/step - loss: 1638.2518 - mse: 1638.2518 - mae: 25.9525
Epoch 147/5000
5/5 [==============================] - 0s 3ms/step - loss: 1632.0907 - mse: 1632.0907 - mae: 26.7047
Epoch 148/5000
5/5 [==============================] - 0s 3ms/step - loss: 1670.9139 - mse: 1670.9139 - mae: 26.6967
Epoch 149/5000
5/5 [==============================] - 0s 3ms/step - loss

5/5 [==============================] - 0s 3ms/step - loss: 1494.2013 - mse: 1494.2013 - mae: 24.6176
Epoch 212/5000
5/5 [==============================] - 0s 3ms/step - loss: 1624.8258 - mse: 1624.8258 - mae: 25.9325
Epoch 213/5000
5/5 [==============================] - 0s 3ms/step - loss: 1628.6605 - mse: 1628.6605 - mae: 25.9688
Epoch 214/5000
5/5 [==============================] - 0s 3ms/step - loss: 1701.1450 - mse: 1701.1450 - mae: 26.2475
Epoch 215/5000
5/5 [==============================] - 0s 2ms/step - loss: 1594.0245 - mse: 1594.0245 - mae: 25.7123
Epoch 216/5000
5/5 [==============================] - 0s 3ms/step - loss: 1525.9509 - mse: 1525.9509 - mae: 25.3082
Epoch 217/5000
5/5 [==============================] - 0s 3ms/step - loss: 1480.7640 - mse: 1480.7639 - mae: 26.1336
Epoch 218/5000
5/5 [==============================] - 0s 3ms/step - loss: 1624.0137 - mse: 1624.0137 - mae: 26.6516
Epoch 219/5000
5/5 [==============================] - 0s 3ms/step - loss: 1569.4550 - m

...

Epoch 1963/5000
10/10 [==============================] - 0s 3ms/step - loss: 1957.2125 - mse: 1957.2125 - mae: 23.6285
Epoch 1964/5000
10/10 [==============================] - 0s 3ms/step - loss: 2150.6123 - mse: 2150.6123 - mae: 23.3102
Epoch 1965/5000
10/10 [==============================] - 0s 3ms/step - loss: 1517.5446 - mse: 1517.5446 - mae: 22.9422
Epoch 1966/5000
10/10 [==============================] - 0s 3ms/step - loss: 1887.7208 - mse: 1887.7208 - mae: 23.9023
Epoch 1967/5000
10/10 [==============================] - 0s 3ms/step - loss: 2317.9651 - mse: 2317.9651 - mae: 24.6828
Epoch 1968/5000
10/10 [==============================] - 0s 3ms/step - loss: 2545.6248 - mse: 2545.6248 - mae: 25.0859
Epoch 1969/5000
10/10 [==============================] - 0s 3ms/step - loss: 1788.4802 - mse: 1788.4802 - mae: 24.9016
Epoch 1970/5000
10/10 [==============================] - 0s 3ms/step - loss: 1590.0488 - mse: 1590.0488 - mae: 24.4362
Epoch 1971/5000
10/10 [=========================

10/10 [==============================] - 0s 3ms/step - loss: 1947.5792 - mse: 1947.5792 - mae: 24.4577
Epoch 2032/5000
10/10 [==============================] - 0s 3ms/step - loss: 1803.4813 - mse: 1803.4813 - mae: 24.5059
Epoch 2033/5000
10/10 [==============================] - 0s 3ms/step - loss: 1851.5168 - mse: 1851.5167 - mae: 24.5698
Epoch 2034/5000
10/10 [==============================] - 0s 3ms/step - loss: 1856.1945 - mse: 1856.1945 - mae: 24.4858
Epoch 2035/5000
10/10 [==============================] - 0s 3ms/step - loss: 2315.3655 - mse: 2315.3655 - mae: 24.7330
Epoch 2036/5000
10/10 [==============================] - 0s 3ms/step - loss: 2095.2537 - mse: 2095.2537 - mae: 23.9912
Epoch 2037/5000
10/10 [==============================] - 0s 2ms/step - loss: 2926.5574 - mse: 2926.5574 - mae: 25.4038
Epoch 2038/5000
10/10 [==============================] - 0s 3ms/step - loss: 1820.2664 - mse: 1820.2665 - mae: 23.8318
Epoch 2039/5000
10/10 [==============================] - 0s 3ms/

Epoch 2100/5000
10/10 [==============================] - 0s 3ms/step - loss: 1449.2325 - mse: 1449.2325 - mae: 23.6510
Epoch 2101/5000
10/10 [==============================] - 0s 3ms/step - loss: 1361.5168 - mse: 1361.5168 - mae: 23.1644
Epoch 2102/5000
10/10 [==============================] - 0s 3ms/step - loss: 1649.2465 - mse: 1649.2465 - mae: 24.4562
Epoch 2103/5000
10/10 [==============================] - 0s 3ms/step - loss: 2067.7737 - mse: 2067.7737 - mae: 24.5475
Epoch 2104/5000
10/10 [==============================] - 0s 3ms/step - loss: 1871.6362 - mse: 1871.6362 - mae: 24.1006
Epoch 2105/5000
10/10 [==============================] - 0s 3ms/step - loss: 2534.4854 - mse: 2534.4854 - mae: 25.5763
Epoch 2106/5000
10/10 [==============================] - 0s 3ms/step - loss: 1636.8699 - mse: 1636.8699 - mae: 23.8516
Epoch 2107/5000
10/10 [==============================] - 0s 3ms/step - loss: 1714.9493 - mse: 1714.9493 - mae: 23.9520
Epoch 2108/5000
10/10 [=========================

10/10 [==============================] - 0s 3ms/step - loss: 1354.6128 - mse: 1354.6128 - mae: 24.0716
Epoch 2169/5000
10/10 [==============================] - 0s 3ms/step - loss: 1879.8906 - mse: 1879.8906 - mae: 24.6203
Epoch 2170/5000
10/10 [==============================] - 0s 3ms/step - loss: 2770.9521 - mse: 2770.9521 - mae: 24.2089
Epoch 2171/5000
10/10 [==============================] - 0s 3ms/step - loss: 1934.0291 - mse: 1934.0291 - mae: 24.3346
Epoch 2172/5000
10/10 [==============================] - 0s 3ms/step - loss: 1878.3948 - mse: 1878.3948 - mae: 24.5740
Epoch 2173/5000
10/10 [==============================] - 0s 3ms/step - loss: 1776.5905 - mse: 1776.5905 - mae: 24.7092
Epoch 2174/5000
10/10 [==============================] - 0s 3ms/step - loss: 2275.8735 - mse: 2275.8735 - mae: 25.8970
Epoch 2175/5000
10/10 [==============================] - 0s 3ms/step - loss: 1632.7230 - mse: 1632.7230 - mae: 25.1789
Epoch 2176/5000
10/10 [==============================] - 0s 3ms/

Epoch 2237/5000
10/10 [==============================] - 0s 3ms/step - loss: 1769.8436 - mse: 1769.8436 - mae: 23.4068
Epoch 2238/5000
10/10 [==============================] - 0s 2ms/step - loss: 1328.3154 - mse: 1328.3154 - mae: 24.7643
Epoch 2239/5000
10/10 [==============================] - 0s 2ms/step - loss: 1567.9185 - mse: 1567.9185 - mae: 24.3896
Epoch 2240/5000
10/10 [==============================] - 0s 3ms/step - loss: 1733.9153 - mse: 1733.9153 - mae: 24.0102
Epoch 2241/5000
10/10 [==============================] - 0s 2ms/step - loss: 1501.8506 - mse: 1501.8506 - mae: 24.3406
Epoch 2242/5000
10/10 [==============================] - 0s 2ms/step - loss: 1345.3684 - mse: 1345.3683 - mae: 23.6680
Epoch 2243/5000
10/10 [==============================] - 0s 3ms/step - loss: 1548.9314 - mse: 1548.9314 - mae: 24.5913
Epoch 2244/5000
10/10 [==============================] - 0s 3ms/step - loss: 1518.1348 - mse: 1518.1348 - mae: 23.4196
Epoch 2245/5000
10/10 [=========================